### Gestire escrow condizionali sulla blockchain XRP Ledger.
Gli escrow condizionali permettono di bloccare fondi che possono essere rilasciati solo quando si soddisfano determinate condizioni, in questo caso definite da un fulfillment crittografico.

- condition: è una "promessa" crittografica, rappresentata come un hash crittografico, che definisce i requisiti per sbloccare l'escrow. È un valore derivato matematicamente da un dato segreto (il fulfillment);

- fulfillment: è la "chiave" crittografica necessaria per soddisfare la condizione e sbloccare l'escrow. È il segreto utilizzato per dimostrare che la condizione è stata rispettata. È il dato originale usato per generare la condizione e dev'essere fornito dal destinatario o dal completatore della transazione.

### Import

In [33]:
import xrpl
from xrpl.clients import JsonRpcClient
from xrpl.wallet import Wallet
from datetime import datetime # per calcolare le date di scadenza
from xrpl.models.transactions import EscrowCreate, EscrowFinish
from os import urandom # per generare casualmente i dati della condizione crittografica
from cryptoconditions import PreimageSha256
from utils import get_wallet

### testnet_url

In [2]:
testnet_url = "https://s.altnet.rippletest.net:51234"

### Funzioni

In [ ]:
# crea un array di byte casuali (32 bit)
def generate_condition(randy=None):
    if randy is None:
        randy = urandom(32)

    fulfillment = PreimageSha256(preimage=randy)  # crea la condizione crittografica

    # restituisce la condizione in formato esadecimale che dev'essere inclusa nella transazione di escrow,
    # nonchè il fulfillment per completare l'escrow
    return (fulfillment.condition_binary.hex().upper(), fulfillment.serialize_binary().hex().upper())

# Praticamente volevo fare reverse engineering della funzione generate_condition(), che utilizza PreimageSha256, mi chiedevo la differenza tra usare PreimageSha256 e semplicemente SHA256. Io con SHA256 avrei considerato semplicemente come condition l'hash di rand e come fulfillment rand stesso, ma ho notato che non coincideva con quello che fa PreimageSha256. Se vai a vedere il codice di PreimageSha256 (in articolare la proprietà condition_binary e il metodo serialize_binary), la differenza principale è che sia per la condition che per il fullfillment viene fatta una "serializzazione", cioè ad un certo punto esce un dizionario e fondamentalmente poi questo dizionario viene encodatto a bytes ed è quello che ho fatto io, ho aggiunto i bytes necessari per la serializzazione che sono sempre fissi, poi il resto era quello che pensavo.
from Crypto.Hash import SHA256


def my_generate_condition(randy=None):
    if randy is None:
        randy = urandom(32)

    h = SHA256.new()
    h.update(randy)
    condition = b"\xa0%\x80 " + h.digest() + b"\x81\x01 "
    fulfillment = b'\xa0"\x80 ' + randy

    condition = condition.hex().upper()
    fulfillment = fulfillment.hex().upper()

    return (condition, fulfillment)


# Compare the two methods
randy = urandom(32)
print(generate_condition(randy))
print(my_generate_condition(randy))

assert generate_condition(randy) == my_generate_condition(randy)

('A0258020EC31E2F5E259E2F2DBDBD7680E7DE5482D6F88DF3807B4A326A68C358DB75615810120', 'A02280205B9CC83DF5E3A58D71D6D06B24FA7186FD195268B1C4360FFCE4567ADEDED3DE')
('A0258020EC31E2F5E259E2F2DBDBD7680E7DE5482D6F88DF3807B4A326A68C358DB75615810120', 'A02280205B9CC83DF5E3A58D71D6D06B24FA7186FD195268B1C4360FFCE4567ADEDED3DE')


In [25]:
# aggiunge un numero di secondi alla data corrente (si parte dal 1 gennaio 2000)
# si fa per generare il timestamp Ripple
def add_seconds(numOfSeconds):
    new_date = datetime.now()
    if new_date != '':
        new_date = xrpl.utils.datetime_to_ripple_time(new_date)
        new_date = new_date + int(numOfSeconds)
    return new_date

In [26]:
# creazione di un Escrow Condizionale
def create_conditional_escrow(seed, amount, destination, cancel, condition):
    wallet = Wallet.from_seed(seed)
    client = JsonRpcClient(testnet_url)
    cancel_date = add_seconds(cancel) # data di scadenza oltre la quale l'escrow può essere cancellato
    #finish_date = cancel_date - 200 # data di completamento impostata 200 secondi prima della scadenza
    
    # creazione della transazione EscrowCreate
    escrow_tx=xrpl.models.transactions.EscrowCreate(
        account=wallet.address,
        amount=amount,
        destination=destination,
        cancel_after=cancel_date,
        condition=condition
    )
    
    # invio della transazione: si attende che venga convalidata
    try:
        print("Invio della transazione EscrowCreate...")
        response = xrpl.transaction.submit_and_wait(escrow_tx, client, wallet)
        print("Escrow creato con successo")
        reply = response.result
    except xrpl.transaction.XRPLReliableSubmissionException as e:
        #reply = f"Submit failed: {e}"
        print(f"Errore nella creazione dell'escrow: {e}")
        return None

In [29]:
# completamento dell'Escrow
def finish_conditional_escrow(seed, owner, sequence, condition, fulfillment):
    wallet = Wallet.from_seed(seed)
    client = JsonRpcClient(testnet_url)
    finish_tx = xrpl.models.transactions.EscrowFinish(
        account=wallet.address,
        owner=owner,
        offer_sequence=int(sequence), # numero di sequenza della transazione EscrowCreate
        condition=condition,
        fulfillment=fulfillment
    )

    # sottomette la transazione e attende il risultato
    reply=""
    try:
        print("Invio della transazione EscrowFinish...")
        response = xrpl.transaction.submit_and_wait(finish_tx,client,wallet)
        print("Escrow completato con successo!")
        reply = response.result
    except xrpl.transaction.XRPLReliableSubmissionException as e:
        #reply=f"Submit failed: {e}"
        #return reply
        print(f"Errore nel completamento dell'escrow: {e}")
        return None

### MAIN

In [ ]:
import nest_asyncio
nest_asyncio.apply()

def main():
    # CONFIGURAZIONE INIZIALE
    print("\n--- Configurazione iniziale ---")
    
    # Wallet sorgente
    seed_sender = ""

    wallet_sender = get_wallet(seed_sender)
    seed_sender = wallet_sender.seed
    print("Generated wallet sender with seed: ", seed_sender)
    print("Address: ", wallet_sender.classic_address)

    # Wallet ricevente
    seed_receiver = ""

    wallet_receiver = get_wallet(seed_receiver)
    seed_receiver = wallet_receiver.seed
    print("Generated wallet 2 with seed: ", seed_receiver)
    print("Address: ", wallet_receiver.classic_address)

    destination = wallet_receiver.address
    amount = "1000"  # Importo da bloccare in drops (1 XRP = 1,000,000 drops)
    cancel_seconds = 3600  # Tempo di annullamento (1 ora)
    
    print(f"Seed wallet mittente: {seed_sender}")
    print(f"Destinatario: {destination}")
    print(f"Importo: {amount} drops")
    print(f"Durata Escrow: {cancel_seconds} secondi")

    # GENERAZIONE CONDIZIONE E FULFILLMENT
    print("\n--- Generazione della condizione crittografica ---")
    condition, fulfillment = generate_condition()
    print(f"Condizione generata: {condition}")
    print(f"Fulfillment generato: {fulfillment}")

    # CREAZIONE ESCROW CONDIZIONALE
    print("\n--- Creazione dell'escrow condizionale ---")
    escrow_response = create_conditional_escrow(seed_sender, amount, destination, cancel_seconds, condition)
    print(f"Risultato creazione escrow: {escrow_response}")

    # VERIFICA TRANSAZIONE ESCROW
    if isinstance(escrow_response, dict) and "Sequence" in escrow_response:
        sequence = escrow_response["Sequence"]
        print(f"Numero di sequenza dell'escrow: {sequence}")
    else:
        print("Errore nella creazione dell'escrow. Terminazione.")
        return

    # ATTESA SIMULATA (es. 10 secondi per test)
    import time
    print("\n--- Attesa prima di completare l'escrow (simulazione)... ---")
    time.sleep(10)

    # COMPLETAMENTO ESCROW
    print("\n--- Completamento dell'escrow condizionale ---")
    finish_response = finish_conditional_escrow(seed_sender, wallet_sender.address, sequence, condition, fulfillment)
    print(f"Risultato completamento escrow: {finish_response}")

    # RISULTATO FINALE
    print("\n--- Risultato finale ---")
    if isinstance(finish_response, dict) and "engine_result" in finish_response:
        print(f"Escrow completato con successo! Risultato: {finish_response['engine_result']}")
    else:
        print("Errore nel completamento dell'escrow.")

if __name__ == "__main__":
    main()


--- Configurazione iniziale ---
Attempting to fund address r3RLeuhWaHTx6qW8ATSqgHwLqY9dsgU1PF
Faucet fund successful.
Generated wallet sender with seed:  sEdTrkjKCrtaDrpPo4jKAXjr7UFakKC
Address:  r3RLeuhWaHTx6qW8ATSqgHwLqY9dsgU1PF
Attempting to fund address rhcy4K75QT9oFBqe7i6JghegwNAFUdGJDU
Faucet fund successful.
Generated wallet 2 with seed:  sEdTv4H6nWpi6nL3CjZEELqfLEQ29qU
Address:  rhcy4K75QT9oFBqe7i6JghegwNAFUdGJDU
Seed wallet mittente: sEdTrkjKCrtaDrpPo4jKAXjr7UFakKC
Destinatario: rhcy4K75QT9oFBqe7i6JghegwNAFUdGJDU
Importo: 1000 drops
Durata Escrow: 3600 secondi

--- Generazione della condizione crittografica ---
Condizione generata: A025802075C57C908095C4DF3D89A348DCE8D6DE20E77F5F66AE2DC0406E0A55CA2E0A45810120
Fulfillment generato: A02280203E4AAC0EF9D5E60F000008DE84C41911ECF37AD37F3DF61C614C93E34B6AFFA8

--- Creazione dell'escrow condizionale ---
Invio della transazione EscrowCreate...
Escrow creato con successo
Risultato creazione escrow: None
Errore nella creazione dell'escr